In [2]:
!git clone -b research https://github.com/phungbminh/graph-data-science.git

Cloning into 'graph-data-science'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 42 (delta 8), reused 39 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 1.02 MiB | 7.65 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
import os
os.chdir("/kaggle/working/graph-data-science")

In [4]:
import torch  

cuda_available = torch.cuda.is_available()  
cuda_version = torch.version.cuda  

print(f"CUDA Available: {cuda_available}")  
print(f"CUDA Version: {cuda_version}")

CUDA Available: True
CUDA Version: 12.3


In [ ]:
!conda install -c rapidsai -c conda-forge -c nvidia nx-cugraph cuda-version=12.3 -y

In [6]:
!conda install -c rapidsai -c conda-forge -c nvidia cugraph cuda-version=12.3 -y

Channels:
 - rapidsai
 - conda-forge
 - nvidia
 - nodefaults
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cuda-version=12.3
    - cugraph


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cugraph-24.08.00           |cuda12_py310_240808_gfc880db0c_0         2.2 MB  rapidsai
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  cugraph            rapidsai/linux-64::cugraph-24.08.00-cuda12_py310_240808_gfc880db0c_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
!pip install ogb
!pip install torch_geometric
!pip install dgl-cu123

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00a 0:00:01
ERROR: Could not find a version that satisfies the requirement dgl-cu123 (from versions: none)
ERROR: No matching distribution found for dgl-cu123


In [8]:
import utils.g_biokg as gbg

In [9]:
G = gbg.createBioKG()

Downloaded 0.90 GB: 100%|██████████| 920/920 [00:21<00:00, 43.23it/s]


Extracting /kaggle/working/graph-data-science/dataset/biokg.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 3256.45it/s]

Saving...


In [10]:
import utils.biokg2df as dfbkg
biokg_edge_list = dfbkg.load_edge_df();
biokg_edge_list

biokg_node_list = dfbkg.load_node_df(edge_index);
import pandas as pd
# dictionary of lists
df = pd.DataFrame(biokg_edge_list)
df.to_csv('edge_index.csv')

NameError: name 'edge_index' is not defined

In [10]:
import numpy as np  
import cudf  
import cugraph  
import networkx as nx  
from IPython.display import SVG  
from sknetwork.visualization import svg_graph  
from scipy.sparse import csr_matrix  

def draw_graph(G, show_names=False, node_size=1, font_size=10, edge_width=0.5):  
    # Convert NetworkX graph to edge list  
    edge_list = nx.to_pandas_edgelist(G)  
    print(edge_list.head())  # Kiểm tra dữ liệu đầu vào  
    
    # Convert to cuDF DataFrame  
    cu_edge_list = cudf.DataFrame({  
        'source': edge_list['source'],  
        'target': edge_list['target'],  
        'weight': edge_list.get('weight', cudf.Series([1.0] * len(edge_list)))  # Default weight to 1.0 if not present  
    })  
    print(cu_edge_list.head())  # Kiểm tra dữ liệu cuDF  
    
    # Create a CuGraph graph  
    g_cugraph = cugraph.Graph()
    g_cugraph.from_cudf_edgelist(cu_edge_list, source='source', destination='target', edge_attr='weight')  

    # Calculate PageRank using CuGraph  
    pagerank_scores = cugraph.pagerank(g_cugraph)  
    print(pagerank_scores.head())  # Kiểm tra kết quả PageRank  
    scores = pagerank_scores['pagerank'].to_numpy()  

    # Prepare for visualization 
    adjacency = nx.adjacency_matrix(G, nodelist=None, dtype=None, weight='weight')  
    names = np.array(list(G.nodes()))  

    # Create a Bunch object for compatibility with svg_graph  
    graph = {  
        'adjacency': csr_matrix(adjacency),  
        'names': np.array(names)
    }  

    if show_names:  
        image = svg_graph(graph['adjacency'], font_size=font_size, node_size=node_size, names=graph['names'], width=700, height=500, scores=scores, edge_width=edge_width)  
    else:  
        image = svg_graph(graph['adjacency'], node_size=node_size, width=700, height=500, scores=scores, edge_width=edge_width)  
    
    return SVG(image)

In [ ]:
draw_graph(G, show_names=True, node_size=2, edge_width=0.2)

In [ ]:
import numpy as np  
import cudf  
import cugraph  
import networkx as nx  
import graphistry  
from matplotlib import cm


def draw_graph_with_graphistry(G, show_names=False, node_size=1, font_size=10, edge_width=0.5):  
    # Đăng nhập vào Graphistry  
    graphistry.register(api=3, protocol='https', server='hub.graphistry.com', username='phungbm.work',
                        password='Bph$ng@2@598')  
    edge_list = nx.to_pandas_edgelist(G)  
    print("Edge List:")  
    print(edge_list)  # Kiểm tra dữ liệu đầu vào  

    # Chuyển đổi sang DataFrame cuDF  
    cu_edge_list = cudf.DataFrame({  
        'source': edge_list['source'],  
        'target': edge_list['target'],  
        'weight': edge_list.get('weight', cudf.Series([1.0] * len(edge_list)))  # Mặc định trọng số là 1.0 nếu không có  
    }) 
    print("cuDF Edge List:")  
    print(cu_edge_list)  # Kiểm tra dữ liệu cuDF  

    # Kiểm tra các cột và kiểu dữ liệu  
    print("Data types:")  
    print(cu_edge_list.dtypes)  

    # Create a CuGraph graph  
    g_cugraph = cugraph.Graph()  
    g_cugraph.from_cudf_edgelist(cu_edge_list, source='source', destination='target', edge_attr='weight')  

    # Calculate PageRank using CuGraph  
    pagerank_scores = cugraph.pagerank(g_cugraph)  
    print("Pagerank Scores:")  
    print(pagerank_scores)  # Kiểm tra kết quả PageRank  
    scores = pagerank_scores['pagerank'].to_numpy()  

    # Generate a list of colors from a colormap  
    colormap = cm.get_cmap('coolwarm', 256)  
    colors = [colormap(i) for i in range(colormap.N)]  
    color_hex = [cm.colors.to_hex(c) for c in colors]  # Convert to hex colors  

    # Sử dụng Graphistry để vẽ đồ thị  
    plot = graphistry.bind(source='source', destination='target', node='node')  # Chỉ định cột node  
    plot = plot.edges(cu_edge_list)  

    # Chỉ định các nút cần thiết  
    node_df_drop_duplicates = node_df[['Node']].rename(columns={'source': 'node'}).drop_duplicates()  # Lấy danh sách nút duy nhất  
    plot = plot.nodes(node_df_drop_duplicates)  
    print("node_df:")  
    print(node_df_drop_duplicates)
    plot = plot.encode_point_size('weight')  
    plot = plot.encode_edge_color('weight', palette=color_hex, as_continuous=True)  
    
    # Hiển thị đồ thị  
    try:  
        plot.plot()  
    except Exception as e:  
        print(f"Error while plotting: {e}")

In [ ]:
import graphistry
import pandas as pd
from matplotlib import cm


# Generate a list of colors from a colormap  
colormap = cm.get_cmap('coolwarm', 256)  
colors = [colormap(i) for i in range(colormap.N)]  
color_hex = [cm.colors.to_hex(c) for c in colors]  # Convert to hex colors  
#4762678
list_t = biokg_edge_list.head(3000000)
# Bind
g1 = graphistry.edges(list_t, 'Origin', 'Destination')
# Override styling defaults
g1_styled = g1.encode_edge_color('EdgeType',color_hex ,as_continuous=True)

#g1_styled = g1.encode_edge_color('weight', palette=color_hex, as_continuous=True)  
# Connect: Free GPU accounts and self-hosting @ graphistry.com/get-started
graphistry.register(api=3, username='phungbm.work', password='Bph$ng@2@598')
# Upload for GPU server visualization session
#g1_styled.plot()
g1_styled.plot()

In [55]:
draw_graph_with_graphistry(G)

Edge List:
               source         target
0        disease_1718   protein_3207
1        disease_1718  protein_12979
2        disease_1718  protein_11623
3        disease_1718   protein_5113
4        disease_1718  protein_16919
...               ...            ...
3320796  protein_4868   protein_7965
3320797  protein_4868   protein_4869
3320798  protein_4868   protein_4867
3320799  protein_4868   protein_8031
3320800  protein_7631   protein_1124

[3320801 rows x 2 columns]
cuDF Edge List:
               source         target  weight
0        disease_1718   protein_3207     1.0
1        disease_1718  protein_12979     1.0
2        disease_1718  protein_11623     1.0
3        disease_1718   protein_5113     1.0
4        disease_1718  protein_16919     1.0
...               ...            ...     ...
3320796  protein_4868   protein_7965     1.0
3320797  protein_4868   protein_4869     1.0
3320798  protein_4868   protein_4867     1.0
3320799  protein_4868   protein_8031     1.0
332080

/opt/conda/lib/python3.10/site-packages/cugraph/structure/symmetrize.py:92: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)
/tmp/ipykernel_30/3985563695.py:41: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colormap = cm.get_cmap('coolwarm', 256)


Pagerank Scores:
       pagerank           vertex
0      0.000012     disease_5211
1      0.000011    protein_16079
2      0.000012     protein_4851
3      0.000011     protein_5651
4      0.000024    function_6336
...         ...              ...
93768  0.000002  sideeffect_2262
93769  0.000002        drug_9373
93770  0.000002        drug_9093
93771  0.000002        drug_8654
93772  0.000002     disease_9459

[93773 rows x 2 columns]
node_df:
                 Node
0        disease_1718
1        disease_4903
2        disease_5480
3        disease_3148
4       disease_10300
...               ...
93768  function_42892
93769  function_37594
93770  function_37624
93771  function_37532
93772  function_37082

[93773 rows x 1 columns]
Error while plotting: 422 Client Error: Unprocessable Entity for url: https://hub.graphistry.com/api/v2/upload/datasets/8f03137723f644099916fbf1c8f31ba2/nodes/arrow


In [9]:
!pip install scikit-network

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.5 MB/s eta 0:00:0000:0100:01


In [ ]:
import networkx as nx
import nx_cugraph as nxcg

nxcg_G = nxcg.from_networkx(G)             # conversion happens once here
nx.betweenness_centrality(nxcg_G, k=1000)  # nxcg Graph type causes cugraph backend
draw_graph(nxcg_G, show_names=True, node_size=2, edge_width=0.2)
